In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [59]:
df = pd.read_csv('22.Ridge_Lasso_ElasticNet/Algerian_forest_fires_dataset_cleaned.csv')

In [60]:
df.head(5)

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,Region
0,1,6,2012,29,57,18,0.0,65.7,3.4,7.6,1.3,3.4,0.5,not fire,0
1,2,6,2012,29,61,13,1.3,64.4,4.1,7.6,1.0,3.9,0.4,not fire,0
2,3,6,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire,0
3,4,6,2012,25,89,13,2.5,28.6,1.3,6.9,0.0,1.7,0.0,not fire,0
4,5,6,2012,27,77,16,0.0,64.8,3.0,14.2,1.2,3.9,0.5,not fire,0


In [61]:
df.drop(columns=['day','month','year','Rain','DC','Temperature','Ws','RH','Classes','Region','DMC'],inplace=True)

In [54]:
df['Classes'] = np.where(df['Classes'].str.contains('not fire'),0,1)

In [62]:
## Divide the features in dependant and independant.
X = df.drop('FWI',axis=1)
y = df['FWI']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=20, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((223, 3), (223,), (20, 3), (20,))

In [63]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

lin_reg = LinearRegression()

lin_reg.fit(X_train_scaled, y_train)
y_pred = lin_reg.predict(X_test_scaled)

mae = mean_absolute_error(y_test,y_pred)
score = r2_score(y_test, y_pred)
adj_r2_score = 1-((1-score)*(len(y_test - 1)/(len(y_test) - len(lin_reg.coef_) - 1)))
lin_reg_intercept = lin_reg.intercept_
lin_reg_coef = lin_reg.coef_

# Constructing the regression equation
feature_names = X_train.columns  # Assuming X_train_scaled is a DataFrame
lin_reg_equation = f"y = {lin_reg_intercept:.4f}"
for coef, feature in zip(lin_reg_coef, feature_names):
    lin_reg_equation += f" + ({coef:.4f} * {feature})"

print("The mean absolute error is: ",mae)
print("The R2 score is: ",score)
print("The adjusted R2 score is:", adj_r2_score)

print("The intercept of the regression is: ",lin_reg_intercept)
print("All the co-efficients of the regression are: ",lin_reg_coef)

print("Regression Equation:", lin_reg_equation)

The mean absolute error is:  0.5520935658964394
The R2 score is:  0.9817435172102745
The adjusted R2 score is: 0.9771793965128431
The intercept of the regression is:  7.190582959641259
All the co-efficients of the regression are:  [-0.75230439  5.21723777  3.58901412]
Regression Equation: y = 7.1906 + (-0.7523 * FFMC) + (5.2172 * ISI) + (3.5890 * BUI)


In [46]:
import statsmodels.api as sm

In [64]:
# Add a constant term for the intercept
X_train_scaled = sm.add_constant(X_train_scaled)
X_test_scaled = sm.add_constant(X_test_scaled)

# Fit the regression model using statsmodels
model = sm.OLS(y_train, X_train_scaled).fit()

print(X_train.columns)

# Get the summary of the regression
print(model.summary())

Index(['FFMC', 'ISI', 'BUI'], dtype='object')
                            OLS Regression Results                            
Dep. Variable:                    FWI   R-squared:                       0.974
Model:                            OLS   Adj. R-squared:                  0.974
Method:                 Least Squares   F-statistic:                     2781.
Date:                Tue, 16 Jul 2024   Prob (F-statistic):          5.30e-174
Time:                        14:36:42   Log-Likelihood:                -358.72
No. Observations:                 223   AIC:                             725.4
Df Residuals:                     219   BIC:                             739.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 